In [1]:
# Imports
import gymnasium as gym
from typing import Callable

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecMonitor
import torch as th
from stable_baselines3.common.callbacks import EvalCallback

import pickle

# Import Our environment
from dev_env import tradingEng



c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load Paths
with open("ZeroCorrFrs1Half","rb") as fp:
    paths1 = pickle.load(fp)

# Load Paths
with open("ZeroCorrFrs2Half","rb") as fp:
    paths1 = paths1 + pickle.load(fp)

with open("ZeroCorrSnd1Half","rb") as fp:
    paths2 = pickle.load(fp)

with open("ZeroCorrSnd2Half","rb") as fp:
    paths2 = paths2 + pickle.load(fp)

with open("ZeroCorrTest","rb") as fp:
    paths_ev = pickle.load(fp)


In [7]:

## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.0005)) #


In [ ]:
# Horizons

# Default
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]),filename='eval_logs-Trainhugeb')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model-hugeb',
    log_path='./logs/eval_logs/ev-hugeb',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

model = PPO("MlpPolicy", envs, batch_size = 252*10*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*20*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=5e6, log_interval=2, callback=eval_callback) 

# 1 yr
#envs = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs', resetdate=1.0),
#    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs',resetdate=1.0)
#]),filename='logs1-Train')
#ev_env = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs',resetdate=1.0),
#]))
#eval_callback = EvalCallback(
#    ev_env,
#    best_model_save_path='./logs/best_model1',
#    log_path='./logs/eval_logs1/ev',
#    eval_freq=252*8*1,
#    deterministic=True,
#    render=False,
#    verbose = True,
#    n_eval_episodes = 8
#)

# Instantiate the agent
#model = PPO("MlpPolicy", envs, batch_size = 252*2*1, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*1, normalize_advantage=True, gamma = 0.9, verbose = 1) 
#
#model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# 20 yr
#envs = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs',resetdate=20.0),
#    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs',resetdate=20.0)
#]),filename='logs20-Train')
#ev_env = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs',resetdate=20.0),
#]))

#eval_callback = EvalCallback(
#    ev_env,
#    best_model_save_path='./logs/best_model20',
#    log_path='./logs/eval_logs20/ev',
#    eval_freq=252*8*20,
#    deterministic=True,
#    render=False,
#    verbose = True,
#    n_eval_episodes = 8
#)

#model = PPO("MlpPolicy", envs, batch_size = 252*2*20, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*20, normalize_advantage=True, gamma = 0.9, verbose = 1) 

#model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

Using cpu device
Eval num_timesteps=20160, episode_reward=-0.09 +/- 0.03
Episode length: 1259.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 1.26e+03     |
|    mean_reward          | -0.0911      |
| time/                   |              |
|    total_timesteps      | 20160        |
| train/                  |              |
|    approx_kl            | 0.0031768065 |
|    clip_fraction        | 0.0221       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.8         |
|    explained_variance   | -0.769       |
|    learning_rate        | 0.00498      |
|    loss                 | -0.000396    |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00109     |
|    std                  | 0.98         |
|    value_loss           | 0.000395     |
------------------------------------------
New best mean reward!
---------------------------------
| rollout/           

In [ ]:
# Big observation
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'small'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'small')
]),filename='logsBigO-train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'small'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigObs',
    log_path='./logs/eval_logsBigO/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

try:
    # Auto Encoder Observation
    envs = VecMonitor(DummyVecEnv([
        lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'auto'),
        lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'auto')
    ]),filename='logsAuto-train')
    ev_env = VecMonitor(DummyVecEnv([
        lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'auto'),
    ]))

    eval_callback = EvalCallback(
        ev_env,
        best_model_save_path='./logs/best_modelAutoObs',
        log_path='./logs/eval_logsAutoO/ev',
        eval_freq=252*8*20,
        deterministic=True,
        render=False,
        verbose = True,
        n_eval_episodes = 3
    )
    model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

    model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 
except:
    print("Autoencoder failed, continuing")





Using cpu device
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
4.997023218892638
Eval num_timesteps=20160, episode_reward=-0.14 +/- 0.04
Episode length: 1259.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1.26e+03    |
|    mean_reward          | -0.144      |
| time/                   |             |
|    total_timesteps      | 20160       |
| train/                  |             |
|    approx_kl            | 0.015343782 |
|    clip_fraction        | 0.0715      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.76       |


In [11]:
# Big Action
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'big', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'big', obs = 'xs')
]),filename='logsBigA-Train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'big', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigAct',
    log_path='./eval_logsBigA/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 



Using cpu device
Eval num_timesteps=20160, episode_reward=-0.09 +/- 0.04
Episode length: 1259.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1.26e+03    |
|    mean_reward          | -0.0884     |
| time/                   |             |
|    total_timesteps      | 20160       |
| train/                  |             |
|    approx_kl            | 0.009682716 |
|    clip_fraction        | 0.147       |
|    clip_range           | 0.2         |
|    entropy_loss         | -56.6       |
|    explained_variance   | 0.0316      |
|    learning_rate        | 0.00498     |
|    loss                 | -0.00138    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00673    |
|    std                  | 0.996       |
|    value_loss           | 0.00274     |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    e

In [ ]:
# Reward l2
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs', reward = '2a'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs', reward = '2a')
]),filename='logsL2-train')

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs', reward = '2a'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelL2',
    log_path='./logs/eval_logsL2/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

Using cpu device
Eval num_timesteps=20160, episode_reward=-0.00 +/- 0.00
Episode length: 1259.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 1.26e+03     |
|    mean_reward          | -0.0012      |
| time/                   |              |
|    total_timesteps      | 20160        |
| train/                  |              |
|    approx_kl            | 0.0032088247 |
|    clip_fraction        | 0.0117       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.83        |
|    explained_variance   | -3.75        |
|    learning_rate        | 0.00498      |
|    loss                 | -0.0014      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00082     |
|    std                  | 0.994        |
|    value_loss           | 0.00263      |
------------------------------------------
New best mean reward!
----------------------------------
| rollout/          

In [ ]:
try:
    policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, use_sde = True, sde_sample_freq = 1260*4)) #
    # Add magnus
    envs = VecMonitor(DummyVecEnv([
        lambda: tradingEng(paths1,action = 'big-Magnus', obs = 'xs'),
        lambda: tradingEng(paths2,action = 'big-Magnus', obs = 'xs')
    ]),filename='logs-TrainMagB')
    ev_env = VecMonitor(DummyVecEnv([
        lambda: tradingEng(paths_ev,action = 'big-Magnus', obs = 'xs'),
    ]))

    eval_callback = EvalCallback(
        ev_env,
        best_model_save_path='./logs/best_modelMagB',
        log_path='./logs/eval_logs/evMagB',
        eval_freq=252*8*15,
        deterministic=True,
        render=False,
        verbose = True,
        n_eval_episodes = 6
    )

    model = PPO("MlpPolicy", envs, batch_size = 252*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

    model.learn(total_timesteps=5e6, log_interval=2, callback=eval_callback) 

    # Add ent and magnus
    envs = VecMonitor(DummyVecEnv([
        lambda: tradingEng(paths1,action = 'small-Magnus', obs = 'xs'),
        lambda: tradingEng(paths2,action = 'small-Magnus', obs = 'xs')
    ]),filename='logs-TrainMagS')
    ev_env = VecMonitor(DummyVecEnv([
        lambda: tradingEng(paths_ev,action = 'small-Magnus', obs = 'xs'),
    ]))

    eval_callback = EvalCallback(
        ev_env,
        best_model_save_path='./logs/best_modelMagS',
        log_path='./logs/eval_logs/evMagS',
        eval_freq=252*8*15,
        deterministic=True,
        render=False,
        verbose = True,
        n_eval_episodes = 6
    )

    model = PPO("MlpPolicy", envs, batch_size = 252*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

    model.learn(total_timesteps=5e6, log_interval=2, callback=eval_callback) 

except:
    print("Magnus failed")

Using cpu device
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.26e+03    |
|    ep_rew_mean          | -0.32       |
| time/                   |             |
|    fps                  | 775         |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 20160       |
| train/                  |             |
|    approx_kl            | 0.006998197 |
|    clip_fraction        | 0.0749      |
|    clip_range           | 0.2         |
|    entropy_loss         | -58.2       |
|    explained_variance   | -1.8        |
|    learning_rate        | 0.00249     |
|    loss                 | -0.0105     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00659    |
|    std                  | 1           |
|    value_loss           | 0.000731    |
-----------------------------------------
-----------------------------------------
| rollout/       

In [ ]:
# Correlations 0.7
#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
with open("0.7Corr1Half.pkl","rb") as fp:
    paths1 = pickle.load(fp)

# Load Paths
with open("0.7Corr2Half.pkl","rb") as fp:
    paths2 = pickle.load(fp)

with open("0.7CorrTest.pkl","rb") as fp:
    paths_ev = pickle.load(fp)

envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]),filename='logs07-train')

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model07',
    log_path='./logs/eval_logs07',
    eval_freq=252*8*5*20,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*4*5, learning_rate=linear_schedule(0.0015), policy_kwargs=policy_kwargs, n_steps=252*8*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=1e7, log_interval=2, callback=eval_callback) 

# Correlations -0.7
#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
with open("NegCorrpt0.pkl","rb") as fp:
    paths1 = pickle.load(fp)
with open("NegCorrpt2.pkl","rb") as fp:
    paths1 = paths1 + pickle.load(fp)
# Load Paths
with open("NegCorrpt4.pkl","rb") as fp:
    paths2 = pickle.load(fp)
with open("NegCorrpt3.pkl","rb") as fp:
    paths2 = paths2 + pickle.load(fp)
with open("n0.7CorrTest.pkl","rb") as fp:
    paths_ev = pickle.load(fp)

envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]),filename='logsm07-train')

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelm07',
    log_path='./logs/eval_logsm07',
    eval_freq=252*8*5*20,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*4*5, learning_rate=linear_schedule(0.0015), policy_kwargs=policy_kwargs, n_steps=252*8*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=1e7, log_interval=2, callback=eval_callback) 





Using cpu device
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.26e+03     |
|    ep_rew_mean          | -0.364       |
| time/                   |              |
|    fps                  | 877          |
|    iterations           | 2            |
|    time_elapsed         | 45           |
|    total_timesteps      | 40320        |
| train/                  |              |
|    approx_kl            | 0.0019163904 |
|    clip_fraction        | 0.00337      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.81        |
|    explained_variance   | -1.2         |
|    learning_rate        | 0.0015       |
|    loss                 | -0.00125     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000381    |
|    std                  | 0.974        |
|    value_loss           | 3.27e-05     |
------------------------------------------
-------------------------------------